In [ ]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    
    return df

def plot_metrics(loss, val_loss):
    fig, (ax1) = plt.subplots(1, 1, sharex='col', figsize=(20,7))
    ax1.plot(loss, label='Train loss')
    ax1.plot(val_loss, label='Validation loss')
    ax1.legend(loc='best')
    ax1.set_title('Loss')
    plt.xlabel('Epochs')

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)

    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))

    return df

In [ ]:
train = load_df("train.csv", nrows=100000)
test = load_df("test.csv", nrows=100000)

In [ ]:
print('TRAIN SET')
print('Rows: %s' % train.shape[0])
print('Columns: %s' % train.shape[1])
print('Features: %s' % train.columns.values)
print()
print('TEST SET')
print('Rows: %s' % test.shape[0])
print('Columns: %s' % test.shape[1])
print('Features: %s' % test.columns.values)

In [ ]:
train = add_time_features(train)
test = add_time_features(test)

# Convert target feature to 'float' type.
train["transactionRevenue"] = train["transactionRevenue"].astype('float')
train['hits'] = train['hits'].astype(float)
test['hits'] = test['hits'].astype(float)
train['pageviews'] = train['pageviews'].astype(float)
test['pageviews'] = test['pageviews'].astype(float)

In [ ]:
# Train
gp_fullVisitorId_train = train.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_train['fullVisitorId'] = gp_fullVisitorId_train.index
gp_fullVisitorId_train['mean_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['hits'].transform('mean')
gp_fullVisitorId_train['mean_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['pageviews'].transform('mean')
gp_fullVisitorId_train['sum_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['hits'].transform('sum')
gp_fullVisitorId_train['sum_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['pageviews'].transform('sum')
gp_fullVisitorId_train = gp_fullVisitorId_train[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
train = train.join(gp_fullVisitorId_train, on='fullVisitorId', how='inner', rsuffix='_')
train.drop(['fullVisitorId_'], axis=1, inplace=True)

# Test
gp_fullVisitorId_test = test.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_test['fullVisitorId'] = gp_fullVisitorId_test.index
gp_fullVisitorId_test['mean_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['hits'].transform('mean')
gp_fullVisitorId_test['mean_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['pageviews'].transform('mean')
gp_fullVisitorId_test['sum_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['hits'].transform('sum')
gp_fullVisitorId_test['sum_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['pageviews'].transform('sum')
gp_fullVisitorId_test = gp_fullVisitorId_test[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
test = test.join(gp_fullVisitorId_test, on='fullVisitorId', how='inner', rsuffix='_')
test.drop(['fullVisitorId_'], axis=1, inplace=True)

In [ ]:
# Drop column that exists only in train data
train.drop(['campaignCode'], axis=1, inplace=True)

# Input missing transactionRevenue values
train["transactionRevenue"].fillna(0, inplace=True)

#For submission file
test_ids = test["fullVisitorId"].values

In [ ]:
# Unwanted columns
unwanted_columns = ['fullVisitorId', 'sessionId', 'visitId', 'visitStartTime', 
                    'browser', 'browserSize', 'browserVersion', 'flashVersion', 
                    'mobileDeviceInfo', 'mobileDeviceMarketingName', 'mobileDeviceModel', 
                    'mobileInputSelector', 'operatingSystemVersion', 'screenColors', 
                    'metro','networkDomain', 'networkLocation', 'adContent', 'campaign', 
                    'isTrueDirect', 'keyword', 'referralPath', 'source', 'operatingSystem', 'day', 'adwordsClickInfo']

train.drop(unwanted_columns, axis=1, inplace=True)
test.drop(unwanted_columns, axis=1, inplace=True)

# Constant columns
constant_columns = [c for c in train.columns if train[c].nunique()<=1]
print('Columns with constant values: ', constant_columns)

train.drop(constant_columns, axis=1, inplace=True)
test.drop(constant_columns, axis=1, inplace=True)

# Columns with more than 50% null data
high_null_columns = [c for c in train.columns if train[c].count()<=len(train) * 0.5]
print('Columns more than 50% null values: ', high_null_columns)
train.drop(high_null_columns, axis=1, inplace=True)
test.drop(high_null_columns, axis=1, inplace=True)

In [ ]:
#Change column types
print(train.dtypes)

In [ ]:
train['isMobile'] = train['isMobile'].astype('object',copy=False)
train['year'] = train['year'].astype('object',copy=False)
train['month'] = train['month'].astype('object',copy=False)
train['weekday'] = train['weekday'].astype('object',copy=False)

In [ ]:
#Change column types
print(test.dtypes)

In [ ]:
test['isMobile'] = test['isMobile'].astype('category',copy=False)
test['year'] = test['year'].astype('category',copy=False)
test['month'] = test['month'].astype('category',copy=False)
test['weekday'] = test['weekday'].astype('category',copy=False)

In [ ]:
class_column = 'transactionRevenue'
numerical_columns = train.select_dtypes(include=[np.number]).columns.drop(class_column)
categorical_columns = train.select_dtypes(include=[np.object]).columns

In [ ]:
def plot_all(df, df_class_name, color='r'):
    for column in df.columns:
        if column in numerical_columns:
            #plt.subplots(figsize=(10,5))
            sns.scatterplot(x=column, y=df_class_name, data=df[[column, df_class_name]])
            plt.show()
        
        if column in categorical_columns:
            #plt.subplots(figsize=(10,5))
            sns.catplot(x=column, y=df_class_name, data=df[[column, df_class_name]], height=8)
            plt.show()
    
plot_all(train, 'transactionRevenue')

In [ ]:
#Concatenate categorical values
train['channelGrouping'].replace(['Paid Search', 'Display', 'Social', 'Affiliates', '(Other)'], 5*['Other'], inplace=True)
train['continent'].replace(['Africa', 'Oceania', '(not set)'], 3*['Other'], inplace=True)
train['medium'].replace(['(none)', 'affiliate', '(not set)'], 3*['Other'], inplace=True)

#Drop multi-valued columns
train.drop(['city', 'country', 'region', 'subContinent'], axis=1, inplace=True)

In [ ]:
#Concatenate categorical values
test['channelGrouping'].replace(['Paid Search', 'Display', 'Social', 'Affiliates', '(Other)'], 5*['Other'], inplace=True)
test['continent'].replace(['Africa', 'Oceania', '(not set)'], 3*['Other'], inplace=True)
test['medium'].replace(['(none)', 'affiliate', '(not set)'], 3*['Other'], inplace=True)

#Drop multi-valued columns
test.drop(['city', 'country', 'region', 'subContinent'], axis=1, inplace=True)

In [ ]:
plot_all(train, 'transactionRevenue')

In [ ]:
print('TRAIN SET')
print('Rows: %s' % train.shape[0])
print('Columns: %s' % train.shape[1])
print('Features: %s' % train.columns.values)
print()
print('TEST SET')
print('Rows: %s' % test.shape[0])
print('Columns: %s' % test.shape[1])
print('Features: %s' % test.columns.values)

In [ ]:
train = pd.get_dummies(train)

In [ ]:
train.columns

In [ ]:
STD_COEFFICIENT = 3
def outlier_analysis(df):
    columns = df.columns
    for column in columns:
        if column in numerical_columns:
            std = df[column].std()
            mean = df[column].mean()
            non_outlier_df = df[(df[column] > mean - STD_COEFFICIENT*std) & (df[column] < mean + STD_COEFFICIENT*std)]
            if len(non_outlier_df) > 0:
                df = non_outlier_df
            else:
                print("column %s  : all elements outlier"%column)
            print("column %s applied"%column)
            print("new lenght: %d" %len(df))
    return df

In [ ]:
outlier_analysis(train.drop(['date','transactionRevenue'], axis=1))

In [ ]:
def calc_correlation(df, df_class):
    for column in df.columns:
        if column in numerical_columns:
            corr = df[column].corr(df_class)
            print("column : %s, corr : %f"%(column, corr))

In [ ]:
calc_correlation(train.drop(['transactionRevenue'], axis=1), train['transactionRevenue'])

In [ ]:
#Drop negative correlation valued columns
train.drop(['sum_hits_per_day', 'sum_pageviews_per_day'], axis=1, inplace=True)
test.drop(['sum_hits_per_day', 'sum_pageviews_per_day'], axis=1, inplace=True)

In [ ]:
numerical_columns = numerical_columns.drop('sum_hits_per_day')
numerical_columns = numerical_columns.drop('sum_pageviews_per_day')

In [ ]:
X_train = train[train['date']<=datetime.date(2017, 5, 31)]
X_val = train[train['date']>datetime.date(2017, 5, 31)]
X_train.drop(['date'], axis=1, inplace=True)
X_val.drop(['date'], axis=1, inplace=True)

In [ ]:
#Get labels
Y_train = X_train['transactionRevenue'].values
Y_val = X_val['transactionRevenue'].values
X_train.drop(['transactionRevenue'], axis=1, inplace=True)
X_val.drop(['transactionRevenue'], axis=1, inplace=True)

In [ ]:
#Log transform the labels
Y_train = np.log1p(Y_train)
Y_val = np.log1p(Y_val)

In [ ]:
reduce_features = ['date']
train.drop(reduce_features, axis=1, inplace=True)
test.drop(reduce_features, axis=1, inplace=True)

In [ ]:
#Normalize numerical columns
scaler = preprocessing.MinMaxScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_val[numerical_columns] = scaler.fit_transform(X_val[numerical_columns])
test[numerical_columns] = scaler.fit_transform(test[numerical_columns])

In [ ]:
#Train

In [ ]:
predictions = model.predict(test)

submission = pd.DataFrame({"fullVisitorId":test_ids})
predictions[predictions<0] = 0
submission["PredictedLogRevenue"] = predictions
submission = submission.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
submission.columns = ["fullVisitorId", "PredictedLogRevenue"]
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"]
submission.to_csv("submission.csv", index=False)